In [1]:
from gensim import corpora

In [ ]:
import pandas as pd
from gensim import corpora
from gensim import models

#######################변경###########################
NUM_TOPICS = 4
NUM_TOPIC_WORDS = 100


######################################################

def get_highest_topic(topic_list):
    highest_topic = 100
    highest_prob = 0
    for topic, prob in topic_list:
        if prob > highest_prob:
            highest_prob = prob
            highest_topic = topic
    return highest_topic, highest_prob


def build_doc_word_matrix(docs):
    dictionary = corpora.Dictionary(docs)   #젠심에서 제공하는 dictionary를 이용해서 전체 데이터에 들어있는 단어들의 고유한 ID를 매김
    corpus = []  #얘는 결국  DTM이 됨. 
    for doc in docs:
        bow = dictionary.doc2bow(doc) #다큐먼트에 사용된 단어가 무엇인지, 또 얼만큼 사용했는지 정보를 이용해서 문서를 벡터로. 
        corpus.append(bow)

    return corpus, dictionary




#########################파일명변경#################################
pre_file_name='C:/hwantajee/TA_buzz_all/TKE_buzz_all_V_utf8.xlsx' 
df = pd.read_excel(pre_file_name) 

#########################분석할POS변경#################################
df=df[(df['pos']=='NNP')|(df['pos']=='NNG')|(df['pos']=='SL+SN')|(df['pos']=='SN+SL')|(df['pos']=='VVVP')|(df['pos']=='VVVX')|(df['pos']=='VVXR')|(df['pos']=='VA+ETM')|(df['pos']=='VA+EC')|(df['pos']=='VA+EF')|(df['pos']=='VA+EP')|(df['pos']=='VAVX')]


unique_uuid_list = df.key.unique().tolist()
total_Noun_words = {}

for u in unique_uuid_list:

    k = (df['key'] == u)
    df_ = df.loc[k,:]
    
    token_list=[]
    content_list = df_['content'].to_list()
    freq_list=df_['tf'].to_list()
    for i,j in list(zip(content_list, freq_list)):
        for a in range(j):
            token_list.append(i)
            
    total_Noun_words[u] = {}

    total_Noun_words[u]['key']= u
    total_Noun_words[u]['words']= token_list
    

    

key= []
documents =[]

for k in total_Noun_words.keys():
    key.append(total_Noun_words[k]['key'])
    documents.append(total_Noun_words[k]['words']) # list of docs: a doc is a list of words
    

    
    
corpus, dictionary = build_doc_word_matrix(documents)



lda_model = models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS,
                        id2word=dictionary,
                        alpha='auto')

#corpus: DTM
#print_topic_words(lda_model)


    

###################################변경####################################
f = open('C:/hwantajee/TA_buzz_all/n_buzz_all_word_topic_4.txt','w') 
for topic_id in range(lda_model.num_topics):
    word_probs = lda_model.show_topic(topic_id, NUM_TOPIC_WORDS)
    #print("Topic ID: {}".format(topic_id))
    #f.write(str(topic_id)+'\n')
    for word, prob in word_probs:
        print("\t{}\t{}\t{}".format(topic_id, word, prob))
        f.write(str(topic_id)+'\t'+str(word)+'\t'+str(prob)+'\n')
    print("\n")
f.close()




#################################변경###########################################
f = open('C:/hwantajee/TA_buzz_all/n_buzz_all_document_topic_4.txt','w')  
for i in range(len(documents)):
    #print(key_id[i])
    topics_list = lda_model.get_document_topics(corpus[i])
    #print(topics_list)
    hi_topic, hi_prob = get_highest_topic(topics_list)
    print(key[i],hi_topic, hi_prob)
    f.write(str(hi_topic)+'\t'+str(key[i])+'\t'+ str(hi_prob)+'\n')
f.close()




In [ ]:
pip install pyLDAvis

In [4]:
import pyLDAvis.gensim_models

In [8]:
import pickle
lda_data = {'corpus': corpus, 'dictionary':dictionary, 'lda_model':lda_model}

#################################변경###########################################
with open('n_news_data.pickle', 'wb') as f:
    pickle.dump(lda_data,f)

    


import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

prepared_data = gensimvis.prepare(lda_model, corpus, dictionary)

#################################변경###########################################
pyLDAvis.save_html(prepared_data, u'nbuzz_all_LDAvis_4.html')